# How to use
分類問題でどのように使用するかを説明します。   

## install
以下のコードをnb_sissoをinstallしたい環境で実行

In [2]:
#!pip install git+https://github.com/souno1218/nb_sisso.git

## データ作成

まず(randomな)データを作成します。   
この時、初期特徴量は`a~f`で6種類(`n_features=6`)、サンプル数は220(`n_samples=220`)、目標変数は`target`としています。   
また、目標変数は2値で、`[True,False]`とします。   

In [1]:
import numpy as np
import pandas as pd
rng = np.random.default_rng()
x=rng.random((6,220))
y=rng.choice((True,False),220)
columns=[f"{chr(i + 97)}" for i in range(x.shape[0])]
index=[f"sample_{i}" for i in range(x.shape[1])]
df=pd.DataFrame(x.T,columns=columns,index=index)
df["target"]=y
df.head()

,a,b,c,d,e,f,target
sample_0,0.058396,0.534238,0.331643,0.796178,0.714435,0.422429,True
sample_1,0.079685,0.105844,0.136312,0.032677,0.417944,0.816955,True
sample_2,0.974836,0.994915,0.521029,0.976663,0.960051,0.434896,False
sample_3,0.814850,0.294788,0.257428,0.073471,0.562043,0.750815,False
sample_4,0.290238,0.285344,0.881876,0.501446,0.862662,0.848639,False


単位系をそれぞれ`{a: "m/s", b: "m^2", c: "m/s^2", d: "a.u.", e: "m", f: "a.u."}`とします。   
この時単位系は2種類なので、以下のようなshapeが`(n_features,2)`なndarrayを作成します。   
この時、dtypeはint64である必要があります。   

In [2]:
units=np.zeros((x.shape[0],2),dtype="int64")
units[0,0] = 1  # a:    "m"
units[0,1] = -1 # a:   "/s"
units[1,0] = 2  # b:  "m^2"
units[1,1] = 0  # b: "no s"
units[2,0] = 1  # c:    "m"
units[2,1] = -2 # c: "/s^2"
units[3,0] = 0  # d: "no m"
units[3,1] = 0  # d: "no s"
units[4,0] = 1  # e:    "m"
units[4,1] = 0  # e: "no s"
units[5,0] = 0  # f: "no m"
units[5,1] = 0  # f: "no s"
units_df=pd.DataFrame(units.T,columns=columns,index=["m","s"])
units_df

,a,b,c,d,e,f
m,1,2,1,0,1,0
s,-1,0,-2,0,0,0


各配列のshapeを見ておきます。

In [3]:
print(f"x.shape={x.shape}, y.shape={y.shape}, units.shape={units.shape}")

x.shape=(6, 220), y.shape=(220,), units.shape=(6, 2)


## SIS

使用する演算子を決めます。以下が使用可能です。   
`["+","-","*","/","*-1","**-1","**2","sqrt","| |","**3","cbrt","**6","exp","exp-","log","sin","cos"]`

In [1]:
operators_to_use=["+","-","*","/","**2","| |","**3"]

model_scoreを決めます。2値分類に関してはさまざま作ってありますが、今のところ多値分類,回帰はありません。   
上のような`x,y`のみを引数に取り、float64の値を2つ返すことができれば、それだけで良いので、pythonがわかればある程度簡単にオリジナルのものを作ることができます。   

In [5]:
from nb_sisso.model_score_1d import Hull_1d
model_score=Hull_1d

SISを走らせます。その際、初期特徴量として`np.ones(n_samples)`を含めるかを`is_use_1`で設定できます。   
`is_use_1 = True`でも、`x`に`np.ones(n_samples)`を加える必要はありません。   
今回は保存数5000(`how_many_to_save=5000`),最大演算子数4(`max_n_op=4`),`is_use_1=True`とします。   
初回実行時はコンパイルが入るため、動き出すまでが遅いです。   
いくつかのオプション引数があります。nb_SO.pyをご覧ください。   

In [6]:
from nb_sisso import SIS
score,eq=SIS(x,y,model_score=model_score,units=units,how_many_to_save=5000,is_use_1=True,max_n_op=4,operators_to_use=operators_to_use)

2024-10-23 11:57:55,085 SIS [INFO] : numba=0.60.0, numpy=2.0.2
2024-10-23 11:57:55,086 SIS [INFO] : OPT=_OptLevel(3), THREADING_LAYER=default
2024-10-23 11:57:55,086 SIS [INFO] : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2024-10-23 11:57:55,086 SIS [INFO] : SIS
2024-10-23 11:57:55,086 SIS [INFO] : num_threads=8, how_many_to_save=5000, 
2024-10-23 11:57:55,087 SIS [INFO] : how_many_to_save_per_1_core=5000, 
2024-10-23 11:57:55,087 SIS [INFO] : max_n_op=4, model_score=Hull_1d, 
2024-10-23 11:57:55,087 SIS [INFO] : x.shape=(6, 220), is_use_1=True
2024-10-23 11:57:55,087 SIS [INFO] : use_binary_op=[-1, -2, -3, -4], 
2024-10-23 11:57:55,087 SIS [INFO] : use_unary_op=[-7, -9, -10]
2024-10-23 11:57:55,088 SIS [INFO] : units=[ 1 -1] , [2 0] , [ 1 -2] , [0 0] , [1 0] , [0 0]
2024-10-23 11:57:55,088 SIS [INFO] : compiling
2024-10-23 11:58:21,407 SIS [INFO] : END, compile
2024-10-23 11:58:21,407 SIS [INFO] :   n_op=1
2024-10-23 11:58:21,606 SIS [INFO] :     binary_op n_op1:n_op2 = 0:0,  lo

返り値は2つで、`score,eq`です。`score`のshapeは`(how_many_to_save,2)`で、`eq`と対応しています。   
scoreは2つあるためdim2は2になっています。   
`eq`のshapeは`(how_many_to_save,2*max_n_op+1)`で、数式を配列にしたもので、このまま読むことはできません。   
`decryption`関数に入れることで、読めるようになります。

In [7]:
from nb_sisso.utils import decryption
decryption(eq[0])

'((((c*f)**2)**3)/b)'

In [8]:
df_SIS_ans=pd.DataFrame()
str_eq=[decryption(eq[i]) for i in range(eq.shape[0])]
df_SIS_ans["score1"]=score[:,0]
df_SIS_ans["score2"]=score[:,1]
df_SIS_ans["str_eq"]=str_eq
df_SIS_ans.head()

,score1,score2,str_eq
0,0.059091,-0.487477,((((c*f)**2)**3)/b)
1,0.059091,-0.487477,((((c*f)**3)**2)/b)
2,0.059091,-0.697063,((((c*c)*f)**3)/b)
3,0.059091,-0.900741,((((e*e)*d)-b)*f)
4,0.059091,-1.833297,np.abs((((e*e)-b)-b))


## SO

SISの結果`how_many_to_save`個から2つ選ぶ組み合わせを全て行い、最もスコアが高いものを選ぶ。   


SISの結果からSO向けの特徴量を作成します。SISで使用した`x`と、SISの返り値の`eq`を`eq_list_to_num`に入れると、`eq`に従って計算された特徴量が返ってきます。   
SISでは特徴量はlistである必要があるため、listに変更します。   

In [9]:
from nb_sisso.utils import eq_list_to_num
X=eq_list_to_num(x,eq)
list_x=[X]

model_scoreを決めます。2値分類に関してはさまざま作ってありますが、今のところ多値分類,回帰はありません。   
上のような`x,y`のみを引数に取り、float64の値を2つ返すことができれば、それだけで良いので、pythonがわかればある程度簡単にオリジナルのものを作ることができます。   

In [10]:
from nb_sisso.model_score_2d import Hull_2d
model_score=Hull_2d

SOを走らせます。`which_arr_to_choose_from`の詳しい役割はnb_SO.pyを見てください。   
`combination_dim`は`how_many_to_save`個からいくつ選ぶ組み合わせを考えるかです。   
いくつかのオプション引数があります。nb_SO.pyをご覧ください。   

In [11]:
from nb_sisso import SO
score_list,index_list=SO(list_x,y,model_score=model_score,which_arr_to_choose_from={1:0,2:0},combination_dim=2)

2024-10-23 11:58:28,410 SO [INFO] : numba=0.60.0, numpy=2.0.2
2024-10-23 11:58:28,411 SO [INFO] : OPT=_OptLevel(3), THREADING_LAYER=default
2024-10-23 11:58:28,412 SO [INFO] : USING_SVML=False, ENABLE_AVX=True, DISABLE_JIT=0
2024-10-23 11:58:28,412 SO [INFO] : SO
2024-10-23 11:58:28,414 SO [INFO] : num_threads=8, how_many_to_save=50, 
2024-10-23 11:58:28,414 SO [INFO] : combination_dim=2, model_score=Hull_2d, 
2024-10-23 11:58:28,414 SO [INFO] : which_arr_to_choose_from={1: 0, 2: 0}
2024-10-23 11:58:28,415 SO [INFO] : loop=12497500
2024-10-23 11:58:28,415 SO [INFO] : compiling
2024-10-23 11:58:36,037 SO [INFO] : END, compile
2024-10-23 11:58:46,038 SO [INFO] :     649903/12497500  0:00:10.000750 : 0:03:02.311600
2024-10-23 11:58:56,050 SO [INFO] :    1291457/12497500  0:00:20.012419 : 0:02:53.648854
2024-10-23 11:59:06,059 SO [INFO] :    1824351/12497500  0:00:30.021926 : 0:02:55.639715
2024-10-23 11:59:16,067 SO [INFO] :    2423975/12497500  0:00:40.029480 : 0:02:46.354013
2024-10-23 

返り値はscoreと、indexになります。indexは入ってきたlist_xのindexです。

In [12]:
arr_columns=np.array([decryption(eq[i]) for i in range(eq.shape[0])])
df_ans=pd.DataFrame(columns=["score1","score2","data_x","data_y"])
df_ans["score1"]=score_list[:,0]
df_ans["score2"]=score_list[:,1]
df_ans["data_x"]=arr_columns[index_list[:,0]]
df_ans["data_y"]=arr_columns[index_list[:,1]]
df_ans

,score1,score2,data_x,data_y
0,0.468182,0.0,(d-(((d+d)+d)**3)),(1/(d+((1-d)**3)))
1,0.450000,0.0,((c/((e**3)**3))**3),(a/(((a/b)*c)**2))
2,0.450000,0.0,((((b/c)*d)**3)**3),(b/(((c*f)**3)**3))
3,0.445455,0.0,((((b/c)*d)**3)**3),((b/((c*f)**3))**3)
4,0.427273,0.0,((((d*e)**3)/b)-e),((((e**3)**3)**3)/d)
5,0.427273,0.0,((((b/e)-e)**2)/b),((b/((e*e)-b))**2)
6,0.427273,0.0,((((b/e)+e)**2)/b),((b/((e*e)-b))**2)
7,0.427273,0.0,(b/(((b/e)+e)**2)),((b/((e*e)-b))**2)
8,0.418182,0.0,((((b/e)-e)**2)/b),(b/np.abs(((e*e)-b)))
9,0.418182,0.0,((((b/e)-e)**2)/b),np.abs((b/((e*e)-b)))
